# Chapter 10: Interfaces – Defining Contracts

Inheritance gives you an "is‑a" relationship and code reuse, but it comes with limitations: a class can inherit from only one base class. What if you need to guarantee that a class supports certain behavior, but those behaviors come from different sources? Enter **interfaces**.

An interface defines a **contract** – a set of methods, properties, events, or indexers that a class or struct **must** implement. Interfaces contain no implementation; they only declare the members. Any class or struct that implements the interface promises to provide the actual code for those members.

In this chapter, you'll learn:

- What interfaces are and why they are fundamental to C# and .NET.
- How to define and implement interfaces.
- The difference between implicit and **explicit interface implementation**.
- How a class can implement **multiple interfaces**.
- **Default interface methods** (C# 8 and later) – interfaces with some implementation.
- When to choose an interface over an abstract class.
- Real‑world examples and best practices.

By the end, you'll be able to design systems based on contracts, enabling loose coupling, testability, and flexibility.

---

## 10.1 What is an Interface?

An interface is like a blueprint for behavior. It declares what members a class should have, but it does not provide the implementation. Think of it as a list of requirements: "If you want to be treated as an `IXyz`, you must have these methods."

### Defining an Interface

Interfaces are declared using the `interface` keyword. By convention, interface names start with a capital **I**.

```csharp
public interface IAnimal
{
    void MakeSound();
    string Name { get; set; }
}
```

This interface requires any implementing class to provide:
- A method `MakeSound` that returns `void` and takes no parameters.
- A property `Name` with both `get` and `set` accessors.

Notice there's no implementation – just signatures.

### Implementing an Interface

A class or struct implements an interface by listing it after a colon (`:`) and providing implementations for all its members.

```csharp
public class Dog : IAnimal
{
    public string Name { get; set; }

    public void MakeSound()
    {
        Console.WriteLine("Woof!");
    }
}

public class Cat : IAnimal
{
    public string Name { get; set; }

    public void MakeSound()
    {
        Console.WriteLine("Meow!");
    }
}
```

Now `Dog` and `Cat` both fulfill the `IAnimal` contract.

### Using Interfaces

You can write code that works with the interface type, not the concrete class. This is the essence of programming to an interface.

```csharp
IAnimal myPet = new Dog();
myPet.Name = "Buddy";
myPet.MakeSound(); // Woof!
```

Because `myPet` is declared as `IAnimal`, you can assign any object that implements `IAnimal`. The actual method called is the one from the object's type (polymorphism again).

### Interfaces Can Contain

- Methods
- Properties
- Events
- Indexers
- (C# 8+) Static members, default implementations, etc.

All interface members are implicitly `public` and cannot have access modifiers (though with default implementations, you can have `private` members to support the default implementation).

---

## 10.2 Implicit vs. Explicit Interface Implementation

By default, when you implement an interface member, it's **implicit** – the member is exposed as a public member of the class. However, you can also implement an interface **explicitly**, which means the member is only accessible through the interface reference, not through the class instance.

### Explicit Interface Implementation

You implement explicitly by writing the interface name before the member name, and you omit the access modifier (it's implicitly private).

```csharp
public interface IFile
{
    void Read();
    void Write();
}

public class Document : IFile
{
    // Explicit implementation
    void IFile.Read()
    {
        Console.WriteLine("Reading document...");
    }

    void IFile.Write()
    {
        Console.WriteLine("Writing document...");
    }

    // Own method
    public void Print()
    {
        Console.WriteLine("Printing document...");
    }
}
```

Now:

```csharp
Document doc = new Document();
doc.Print(); // OK
// doc.Read(); // Error: Read is not accessible on Document

IFile file = doc;
file.Read(); // OK
file.Write(); // OK
```

### Why Use Explicit Implementation?

1. **Resolving name conflicts** – when two interfaces have the same method signature, you can implement them separately.
2. **Hiding implementation details** – you may not want certain members to be part of the class's public API, only available when used through the interface.
3. **Providing different implementations** – you can give different implementations for the same method from different interfaces.

### Example: Two Interfaces with Same Method

```csharp
public interface IWriter
{
    void Write(string message);
}

public interface ILogger
{
    void Write(string message);
}

public class FileLogger : IWriter, ILogger
{
    // Explicit implementation for IWriter
    void IWriter.Write(string message)
    {
        Console.WriteLine($"IWriter: {message}");
    }

    // Explicit implementation for ILogger
    void ILogger.Write(string message)
    {
        Console.WriteLine($"ILogger: {message}");
    }

    // Optional: a public Write method that calls one of them or does something else
    public void Write(string message)
    {
        ((IWriter)this).Write(message); // or choose
    }
}
```

Now you can call the appropriate one by casting to the desired interface.

---

## 10.3 Multiple Interface Inheritance

One of the biggest advantages of interfaces over class inheritance is that a class can implement **multiple interfaces**. This allows a class to fulfill several contracts simultaneously.

```csharp
public interface IReadable
{
    void Read();
}

public interface IWritable
{
    void Write();
}

public class File : IReadable, IWritable
{
    public void Read()
    {
        Console.WriteLine("Reading file...");
    }

    public void Write()
    {
        Console.WriteLine("Writing file...");
    }
}
```

A class can implement any number of interfaces. This is a form of multiple inheritance of behavior (but not of state, because interfaces have no fields).

### Interfaces Can Inherit from Other Interfaces

Interfaces themselves can inherit from one or more interfaces, combining contracts.

```csharp
public interface IReadable
{
    void Read();
}

public interface IStorable : IReadable
{
    void Save();
}

public class Document : IStorable
{
    public void Read() { ... }
    public void Save() { ... }
}
```

Now `Document` must implement both `Read` and `Save`. Interface inheritance allows you to build richer contracts from simpler ones.

---

## 10.4 Default Interface Methods (C# 8+)

Before C# 8, interfaces could not contain any implementation. Starting with C# 8, you can provide **default implementations** for interface members. This allows you to add new members to an interface without breaking existing implementers.

```csharp
public interface ILogger
{
    void Log(string message);

    // Default implementation
    void LogError(string error)
    {
        Log($"ERROR: {error}");
    }
}

public class ConsoleLogger : ILogger
{
    public void Log(string message)
    {
        Console.WriteLine(message);
    }
    // LogError is inherited from the interface – no need to implement
}

// Usage
ILogger logger = new ConsoleLogger();
logger.Log("Info");
logger.LogError("Something went wrong"); // calls default implementation
```

The implementing class can still override the default method by providing its own implementation. Default methods are useful for evolving interfaces without breaking existing code, but they should be used sparingly – interfaces are primarily contracts.

**Important:** Default implementations are only available when the member is accessed through the interface. If you access it through the class type (if the class doesn't provide its own implementation), you won't see the default method.

---

## 10.5 Interfaces vs. Abstract Classes

Both interfaces and abstract classes can define contracts, but they serve different purposes. Here's a comparison:

| Feature | Interface | Abstract Class |
|---------|-----------|----------------|
| Multiple inheritance | A class can implement many interfaces. | A class can inherit only one abstract class. |
| Implementation | Can have default methods (C# 8+), but no fields (except static). | Can have fields, constructors, and full method implementations. |
| Access modifiers | Members are implicitly public. | Members can have any access modifier. |
| State | Cannot hold instance state (fields). | Can hold state (fields). |
| Versioning | Adding a member breaks implementers (unless default implementation is provided). | Adding a member can be done with a default implementation in the base class without breaking derived classes. |
| When to use | To define a contract that multiple unrelated types can fulfill. | To provide a common base with shared code and state for closely related types. |

**Guideline:** Use interfaces when you want to define a capability that can be added to any class, regardless of its position in the inheritance hierarchy. Use abstract classes when you have a family of related types that share code and state.

---

## 10.6 Putting It All Together: A Practical Example

Let's build a simple notification system using interfaces. We'll define an `INotifier` interface, implement it for email and SMS, and then use dependency injection to send notifications.

```csharp
using System;
using System.Collections.Generic;

namespace InterfaceDemo
{
    // Interface defining the contract for notifiers
    public interface INotifier
    {
        void Send(string message);
        string ServiceName { get; }
    }

    // Email implementation
    public class EmailNotifier : INotifier
    {
        public string ServiceName => "Email";

        public void Send(string message)
        {
            // Simulate sending email
            Console.WriteLine($"[Email] Sending: {message}");
        }

        // Own method
        public void AttachFile(string filePath)
        {
            Console.WriteLine($"Attaching {filePath} to email");
        }
    }

    // SMS implementation
    public class SmsNotifier : INotifier
    {
        public string ServiceName => "SMS";

        public void Send(string message)
        {
            // Simulate sending SMS
            Console.WriteLine($"[SMS] Sending: {message}");
        }
    }

    // A class that uses multiple interfaces
    public interface ILoggable
    {
        void Log(string message);
    }

    public class PushNotifier : INotifier, ILoggable
    {
        public string ServiceName => "Push Notification";

        public void Send(string message)
        {
            Console.WriteLine($"[Push] Sending: {message}");
            Log($"Sent push: {message}"); // internal logging
        }

        public void Log(string message)
        {
            Console.WriteLine($"[Log] {message}");
        }
    }

    // Demonstration of explicit interface implementation
    public interface IEncryptor
    {
        string Encrypt(string data);
    }

    public interface ICompressor
    {
        byte[] Compress(byte[] data);
    }

    public class SecureNotifier : INotifier, IEncryptor, ICompressor
    {
        // INotifier implementation (explicit? let's do implicit for simplicity)
        public string ServiceName => "SecureNotifier";

        public void Send(string message)
        {
            string encrypted = ((IEncryptor)this).Encrypt(message);
            Console.WriteLine($"[Secure] Sending encrypted: {encrypted}");
        }

        // Explicit implementation for IEncryptor
        string IEncryptor.Encrypt(string data)
        {
            // Dummy encryption
            return Convert.ToBase64String(System.Text.Encoding.UTF8.GetBytes(data));
        }

        // Explicit implementation for ICompressor
        byte[] ICompressor.Compress(byte[] data)
        {
            // Dummy compression (just return as is)
            Console.WriteLine("Compressing data...");
            return data;
        }
    }

    class Program
    {
        static void Main()
        {
            // Polymorphic collection of notifiers
            List<INotifier> notifiers = new List<INotifier>
            {
                new EmailNotifier(),
                new SmsNotifier(),
                new PushNotifier(),
                new SecureNotifier()
            };

            string message = "Hello, world!";

            // Send message through all notifiers
            foreach (INotifier notifier in notifiers)
            {
                Console.WriteLine($"Using {notifier.ServiceName}:");
                notifier.Send(message);
                Console.WriteLine();
            }

            // Accessing a specific implementation's own method
            EmailNotifier email = new EmailNotifier();
            email.Send("Test");
            email.AttachFile("document.pdf");

            // Using explicit interface implementation
            SecureNotifier secure = new SecureNotifier();
            secure.Send("Secret"); // works

            // To call explicit interface members, cast to the interface
            IEncryptor encryptor = secure;
            string encrypted = encryptor.Encrypt("data");
            Console.WriteLine($"Encrypted: {encrypted}");

            // ICompressor compressor = secure; etc.
        }
    }
}
```

**Explanation:**

- `INotifier` defines the contract for anything that can send a notification. It includes a property `ServiceName` and a method `Send`.
- `EmailNotifier`, `SmsNotifier`, `PushNotifier`, and `SecureNotifier` implement `INotifier` in different ways.
- `PushNotifier` also implements `ILoggable`, showing multiple interface implementation.
- `SecureNotifier` uses **explicit interface implementation** for `IEncryptor` and `ICompressor` to hide those methods from the public API of the class; they're only accessible through the interface references.
- In `Main`, we use polymorphism to send the same message through all notifiers, demonstrating that code can work with the interface without caring about the concrete type.
- We also show how to access explicit interface members by casting.

This example illustrates real‑world usage: defining contracts, implementing them in various classes, and writing flexible code that depends on abstractions rather than concretions.

---

## 10.7 Common Pitfalls and Best Practices

### 1. Program to an Interface, Not an Implementation

This famous design principle means you should declare variables, parameters, and return types as interfaces rather than concrete classes whenever possible. This makes your code more flexible and easier to test.

```csharp
// Good
public void ProcessData(INotifier notifier) { ... }

// Less flexible
public void ProcessData(EmailNotifier notifier) { ... }
```

### 2. Keep Interfaces Focused (Interface Segregation Principle)

Interfaces should be small and focused. A class should not be forced to implement methods it doesn't need. If an interface grows too large, split it into smaller, more specific interfaces.

```csharp
// Avoid
public interface IWorker
{
    void Work();
    void Eat();
    void Sleep();
}

// Better
public interface IWorkable { void Work(); }
public interface IEatable { void Eat(); }
public interface ISleepable { void Sleep(); }
```

### 3. Use Explicit Implementation to Avoid Name Clashes

When implementing multiple interfaces that have members with the same signature, use explicit implementation to provide separate implementations.

### 4. Consider Default Interface Methods for API Evolution

If you own an interface that is widely implemented, adding a new member would break all existing implementers. A default implementation can mitigate this, but it's still a compromise – prefer designing stable interfaces from the start.

### 5. Don't Overuse Interfaces

Not every class needs an interface. If you have a single concrete class that isn't likely to have alternatives, and you don't need to mock it for testing, adding an interface may be unnecessary complexity. Use interfaces when you have multiple implementations or when you need to decouple components.

### 6. Name Interfaces Clearly

Use adjectives or noun phrases that describe the capability: `IEnumerable`, `IDisposable`, `INotifyPropertyChanged`. Avoid generic names like `IInterface`.

### 7. Interfaces Can Derive from Other Interfaces

Use interface inheritance to build more specific contracts, but be mindful of the resulting complexity. Deep hierarchies can be hard to follow.

### 8. Testability

Interfaces are crucial for unit testing because they allow you to create mock implementations. For example, if your class depends on `INotifier`, you can inject a mock notifier in tests.

### 9. Prefer Interfaces Over Abstract Classes for Multiple Inheritance

If you need to combine behaviors from multiple sources, interfaces are the only way. Use abstract classes when you have a single base with shared implementation.

### 10. Document the Contract

Especially for public interfaces, provide XML comments explaining what each member does, what parameters mean, and what exceptions might be thrown. This helps implementers know what to do.

---

## 10.8 Chapter Summary

In this chapter, you've learned about interfaces – one of the most powerful tools in C# for defining contracts and enabling polymorphism across unrelated types:

- **Interfaces** declare a set of members that implementing classes must provide.
- **Implicit implementation** makes the members part of the class's public API.
- **Explicit implementation** hides members behind the interface, useful for resolving conflicts or hiding implementation details.
- A class can implement **multiple interfaces**, giving it multiple capabilities.
- **Default interface methods** (C# 8+) allow interfaces to provide a default implementation, useful for evolving APIs.
- Interfaces vs. abstract classes – each has its place; interfaces are for contracts, abstract classes for shared base implementation.

Interfaces are everywhere in .NET – from `IEnumerable` and `IDisposable` to custom interfaces you'll create in your own applications. Mastering them is essential for writing flexible, testable, and maintainable code.

In the next chapter, **Encapsulation & Best Practices**, we'll revisit encapsulation in depth, exploring how to combine access modifiers, properties, and interfaces to build robust, well‑encapsulated classes. You'll also learn about principles like information hiding, cohesion, and coupling, and how they guide your design decisions.

**Exercises:**

1. Define an interface `IVehicle` with methods `Start()`, `Stop()`, and a property `Speed`. Implement it in `Car` and `Bicycle` classes. Write a method that accepts an `IVehicle` and calls `Start` and `Stop`.
2. Create two interfaces `IPlayable` and `IRecordable` with a common method `Start`. Implement both in a class `MediaDevice` using explicit interface implementation. Demonstrate calling each.
3. Add a default method `Pause()` to `IPlayable` with a default implementation. Test it in a class that doesn't override it.
4. Build a simple logging system with an `ILogger` interface. Implement `ConsoleLogger` and `FileLogger`. Write a class that uses `ILogger` via constructor injection.
5. Refactor a previous exercise (e.g., the employee hierarchy) to use interfaces for specific capabilities (e.g., `IPayable`, `IBenefitsEligible`). Compare with the inheritance approach.

Now, get ready to deepen your understanding of encapsulation in Chapter 11!